In [237]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType,DoubleType,FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import lit
import locale
import sys
import os
import re

In [238]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [239]:
spark

# Leyendo lista de Colegios y Estudiantes

In [240]:
base = '../Colegios'
base2 = '../SB'
colegios = os.listdir('../Colegios')
estudiantes = os.listdir('../SB')

In [241]:
estudiantes.sort()
colegios.sort()
estuDic = {}
coleDic = {}
for estu in estudiantes:
    estuDic[re.findall('\d+', estu)[1]] = estu

for colegio in colegios:
    coleDic[re.findall('\d+', colegio)[1]] = colegio

In [242]:
# Funciòn para unir cualquier nùmero de dataframes con la misma estructura 
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [243]:
print("Archivos de estudiantes: ", estuDic['20121'],estuDic['20122'])
print("Archivos de colegios: ",coleDic['2012'])

Archivos de estudiantes:  SB11_20121.txt SB11_20122.txt
Archivos de colegios:  SB11-CLASIFI-PLANTELES-2012.txt


In [244]:
# SE leen los dos archivos y se crean dos dataframes
df1 = spark.read.load(base2 + '/' + estuDic['20121'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')
df2 = spark.read.load(base2 + '/' + estuDic['20122'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='¬')

In [245]:
df1.columns

['ESTU_TIPODOCUMENTO',
 'ESTU_NACIONALIDAD',
 'ESTU_GENERO',
 'ESTU_FECHANACIMIENTO',
 'PERIODO',
 'ESTU_CONSECUTIVO',
 'ESTU_ESTUDIANTE',
 'ESTU_PAIS_RESIDE',
 'ESTU_ETNIA',
 'ESTU_LIMITA_MOTRIZ',
 'ESTU_LIMITA_INVIDENTE',
 'ESTU_LIMITA_CONDICIONESPECIAL',
 'ESTU_LIMITA_SORDOCONINTERPRETE',
 'ESTU_LIMITA_SORDOSININTERPRETE',
 'ESTU_LIMITA_SORDOCEGUERA',
 'ESTU_DEPTO_RESIDE',
 'ESTU_COD_RESIDE_DEPTO',
 'ESTU_MCPIO_RESIDE',
 'ESTU_COD_RESIDE_MCPIO',
 'ESTU_AREARESIDE',
 'ESTU_MESTERMINOBACHILLER',
 'ESTU_ANOTERMINOBACHILLER',
 'ESTU_VALORPENSIONCOLEGIO',
 'ESTU_OCUPACION_ACTUAL',
 'ESTU_MOTIVO_PRESENTAVALIDACION',
 'ESTU_VECESPRESENTOEXAMEN',
 'ESTU_FECHAGRADOBACHILLER',
 'FAMI_EDUCACIONPADRE',
 'FAMI_EDUCACIONMADRE',
 'FAMI_OCUPACIONPADRE',
 'FAMI_OCUPACIONMADRE',
 'FAMI_ESTRATOVIVIENDA',
 'FAMI_NIVELSISBEN',
 'FAMI_PERSONASHOGAR',
 'FAMI_CUARTOSHOGAR',
 'FAMI_PISOSHOGAR',
 'FAMI_TIENEINTERNET',
 'FAMI_TIENE_SERVICIOTV',
 'FAMI_TIENECOMPUTADOR',
 'FAMI_TIENELAVADORA',
 'FAMI_TIENEMICRO

In [246]:
df2.columns

['ESTU_TIPODOCUMENTO',
 'ESTU_NACIONALIDAD',
 'ESTU_GENERO',
 'ESTU_FECHANACIMIENTO',
 'PERIODO',
 'ESTU_CONSECUTIVO',
 'ESTU_ESTUDIANTE',
 'ESTU_PAIS_RESIDE',
 'ESTU_DEPTO_RESIDE',
 'ESTU_COD_RESIDE_DEPTO',
 'ESTU_MCPIO_RESIDE',
 'ESTU_COD_RESIDE_MCPIO',
 'ESTU_AREARESIDE',
 'ESTU_MESTERMINOBACHILLER',
 'ESTU_ANOTERMINOBACHILLER',
 'ESTU_VALORPENSIONCOLEGIO',
 'ESTU_MOTIVO_PRESENTAVALIDACION',
 'ESTU_FECHAGRADOBACHILLER',
 'FAMI_EDUCACIONPADRE',
 'FAMI_EDUCACIONMADRE',
 'FAMI_OCUPACIONPADRE',
 'FAMI_OCUPACIONMADRE',
 'FAMI_ESTRATOVIVIENDA',
 'FAMI_NIVELSISBEN',
 'FAMI_PERSONASHOGAR',
 'FAMI_CUARTOSHOGAR',
 'FAMI_PISOSHOGAR',
 'FAMI_TIENEINTERNET',
 'FAMI_TIENE_SERVICIOTV',
 'FAMI_TIENECOMPUTADOR',
 'FAMI_TIENELAVADORA',
 'FAMI_TIENEMICROONDAS',
 'FAMI_TIENEHORNO',
 'FAMI_TIENEAUTOMOVIL',
 'FAMI_TIENEDVD',
 'FAMI_TIENE_NEVERA',
 'FAMI_TIENE_CELULAR',
 'FAMI_TELEFONO',
 'FAMI_INGRESOFMILIARMENSUAL',
 'ESTU_TRABAJAACTUALMENTE',
 'ESTU_HORASSEMANATRABAJA',
 'ESTU_ANTECEDENTES',
 'ESTU_ANO

In [247]:
columnasPuntos20121 = [
 'COLE_COD_DANE_ESTABLECIMIENTO',
 'COLE_NOMBRE_ESTABLECIMIENTO',
 'PUNT_MATEMATICAS',
 'PUNT_INGLES',
 'PUNT_CIENCIAS_SOCIALES',
 'PUNT_BIOLOGIA',
 'PUNT_FILOSOFIA',
 'PUNT_FISICA',
 'PUNT_QUIMICA',
 'PUNT_LENGUAJE',
 'PUNT_INTERDISC_MEDIOAMBIENTE',
 'PUNT_INTERDISC_VIOLENCIAYSOC',
 'PUNT_PROFUNDIZA_BIOLOGIA',
 'PUNT_PROFUNDIZA_CSOCIALES',
 'PUNT_PROFUNDIZA_LENGUAJE',
 'PUNT_PROFUNDIZA_MATEMATICA']

columnasPuntos20122 = [
 'COLE_COD_DANE_ESTABLECIMIENTO',
 'COLE_NOMBRE_ESTABLECIMIENTO',
 'PUNT_MATEMATICAS',
 'PUNT_INGLES',
 'PUNT_CIENCIAS_SOCIALES',
 'PUNT_BIOLOGIA',
 'PUNT_FILOSOFIA',
 'PUNT_FISICA',
 'PUNT_QUIMICA',
 'PUNT_LENGUAJE',
 'PUNT_INTERDISC_MEDIOAMBIENTE',
 'PUNT_INTERDISC_VIOLENCIAYSOC',
 'PUNT_PROFUNDIZA_BIOLOGIA',
 'PUNT_PROFUNDIZA_CSOCIALES',
 'PUNT_PROFUNDIZA_LENGUAJE',
 'PUNT_PROFUNDIZA_MATEMATICA',   
]

In [248]:
# SE leen los dos archivos y se crean dos dataframes
df1 = df1.select(columnasPuntos20121)
df2 = df2.select(columnasPuntos20122)

In [249]:
#dft = df1.columns.remove('cole_cod_dane_institucion')
cols = columnasPuntos20122
cols.remove('COLE_COD_DANE_ESTABLECIMIENTO')
cols.remove('COLE_NOMBRE_ESTABLECIMIENTO')

In [250]:
# SE unen los dos dataframes
df = unionAll(df1,df2)
for cp in cols:
    df = df.withColumn(cp, regexp_replace(cp, ',', '.'))
    df = df.withColumn(cp, df[cp].cast("float"))
    df = df.na.fill({cp:0})

In [251]:
print(df.printSchema())

root
 |-- COLE_COD_DANE_ESTABLECIMIENTO: long (nullable = true)
 |-- COLE_NOMBRE_ESTABLECIMIENTO: string (nullable = true)
 |-- PUNT_MATEMATICAS: float (nullable = false)
 |-- PUNT_INGLES: float (nullable = false)
 |-- PUNT_CIENCIAS_SOCIALES: float (nullable = false)
 |-- PUNT_BIOLOGIA: float (nullable = false)
 |-- PUNT_FILOSOFIA: float (nullable = false)
 |-- PUNT_FISICA: float (nullable = false)
 |-- PUNT_QUIMICA: float (nullable = false)
 |-- PUNT_LENGUAJE: float (nullable = false)
 |-- PUNT_INTERDISC_MEDIOAMBIENTE: float (nullable = false)
 |-- PUNT_INTERDISC_VIOLENCIAYSOC: float (nullable = false)
 |-- PUNT_PROFUNDIZA_BIOLOGIA: float (nullable = false)
 |-- PUNT_PROFUNDIZA_CSOCIALES: float (nullable = false)
 |-- PUNT_PROFUNDIZA_LENGUAJE: float (nullable = false)
 |-- PUNT_PROFUNDIZA_MATEMATICA: float (nullable = false)

None


In [252]:
df = df.withColumn('COLE_COD_DANE_ESTABLECIMIENTO', df['COLE_COD_DANE_ESTABLECIMIENTO'].cast("string"))

In [253]:
print("Numero de filas ", df.count())

Numero de filas  580551


In [254]:
# Contando el numero de estudiantes por instituciòn y mostrandola de forma descendente
dsCountEst = df.groupBy('COLE_COD_DANE_ESTABLECIMIENTO').count().orderBy('count',ascending=
False)

In [255]:
dsCountEst = dsCountEst.withColumnRenamed("count","num_est")
dsCountEst.show(10,False)
# Numero de estudiantes por instituciòn

+-----------------------------+-------+
|COLE_COD_DANE_ESTABLECIMIENTO|num_est|
+-----------------------------+-------+
|311001105391                 |1202   |
|105001000108                 |1093   |
|311001105863                 |1076   |
|null                         |961    |
|105001013340                 |831    |
|105615000236                 |660    |
|305001003963                 |659    |
|176001005813                 |620    |
|105154000301                 |594    |
|168001003591                 |586    |
+-----------------------------+-------+
only showing top 10 rows



In [256]:
rowmax = dsCountEst.agg({"num_est": "max"}).collect()[0]
rowmin = dsCountEst.agg({"num_est": "min"}).collect()[0]

num_est_max = rowmax["max(num_est)"]
num_est_min = rowmin["min(num_est)"]

In [257]:
print("Nùmero de estudiantes màximo: ", num_est_max)
print("Nùmero de estudiantes mìnimo: ", num_est_min)

Nùmero de estudiantes màximo:  1202
Nùmero de estudiantes mìnimo:  1


In [258]:

#Se crea una columna con el total de los puntajes    
df= df.withColumn("Suma",df['PUNT_MATEMATICAS']+df['PUNT_INGLES']+df['PUNT_CIENCIAS_SOCIALES']+
                     df['PUNT_BIOLOGIA']+df['PUNT_FILOSOFIA']+df['PUNT_FISICA']+df['PUNT_QUIMICA']+
                     df['PUNT_LENGUAJE']+df['PUNT_INTERDISC_MEDIOAMBIENTE']+df['PUNT_INTERDISC_VIOLENCIAYSOC']+
                     df['PUNT_PROFUNDIZA_BIOLOGIA']+
                     df['PUNT_PROFUNDIZA_CSOCIALES']+df['PUNT_PROFUNDIZA_LENGUAJE']+df['PUNT_PROFUNDIZA_MATEMATICA'])


## Resultados finales por instituciòn

In [259]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

df = df.groupBy('COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO').mean()

rowprommax = df.agg({"avg(Suma)": "max"}).collect()[0]
rowprommin = df.agg({"avg(Suma)": "min"}).collect()[0]

prom_est_max = rowprommax["max(avg(Suma))"]
prom_est_min = rowprommin["min(avg(Suma))"]

print("Màxima calificaciòn en colegios: ", prom_est_max)
print("Mìxima calificaciòn en colegios: ", prom_est_min)

Màxima calificaciòn en colegios:  586.6666666666666
Mìxima calificaciòn en colegios:  283.6363636363636


## Creación de Indices de evaluación
$Indice\_Prom = \frac{avg(Suma)}{max(avg(Suma))}$&nbsp;&nbsp;
$0 < Indice\_Prom \leq 1$

$Indice\_Num = \frac{num\_est}{max(num\_est)}$&nbsp;&nbsp;
$0 < Indice\_Num \leq 1$

$Indice\_Total = Indice\_Prom \times Indice\_Num$&nbsp;&nbsp;
$0 < Indice\_Total \leq 1 $

$Indice\_Total$: tiene en consideraciòn el promedio de las calificaciones y el nùmero de estudiantes 



In [260]:
dfjoin = df.join(dsCountEst,"COLE_COD_DANE_ESTABLECIMIENTO")
dfjoin= dfjoin.withColumn("Indice_Prom",round(dfjoin['avg(Suma)']/prom_est_max,3))
dfjoin= dfjoin.withColumn("Indice_Num",round(dfjoin['num_est']/num_est_max,3))
dfjoin= dfjoin.withColumn("Indice_Total",round(dfjoin['Indice_Prom']*dfjoin['Indice_Num'],3))

In [261]:
dfjoin.printSchema()

root
 |-- COLE_COD_DANE_ESTABLECIMIENTO: string (nullable = true)
 |-- COLE_NOMBRE_ESTABLECIMIENTO: string (nullable = true)
 |-- avg(PUNT_MATEMATICAS): double (nullable = true)
 |-- avg(PUNT_INGLES): double (nullable = true)
 |-- avg(PUNT_CIENCIAS_SOCIALES): double (nullable = true)
 |-- avg(PUNT_BIOLOGIA): double (nullable = true)
 |-- avg(PUNT_FILOSOFIA): double (nullable = true)
 |-- avg(PUNT_FISICA): double (nullable = true)
 |-- avg(PUNT_QUIMICA): double (nullable = true)
 |-- avg(PUNT_LENGUAJE): double (nullable = true)
 |-- avg(PUNT_INTERDISC_MEDIOAMBIENTE): double (nullable = true)
 |-- avg(PUNT_INTERDISC_VIOLENCIAYSOC): double (nullable = true)
 |-- avg(PUNT_PROFUNDIZA_BIOLOGIA): double (nullable = true)
 |-- avg(PUNT_PROFUNDIZA_CSOCIALES): double (nullable = true)
 |-- avg(PUNT_PROFUNDIZA_LENGUAJE): double (nullable = true)
 |-- avg(PUNT_PROFUNDIZA_MATEMATICA): double (nullable = true)
 |-- avg(Suma): double (nullable = true)
 |-- num_est: long (nullable = false)
 |-- Indice

In [262]:
dfjoin = dfjoin.select('COLE_COD_DANE_ESTABLECIMIENTO','COLE_NOMBRE_ESTABLECIMIENTO','avg(Suma)','num_est','Indice_Prom','Indice_Num','Indice_Total')
dfjoin = dfjoin.withColumn("rankProm", dense_rank().over(Window.orderBy(desc("Indice_Prom"))))
dfjoin = dfjoin.withColumn("rankNum", dense_rank().over(Window.orderBy(desc("Indice_Num"))))
dfjoin = dfjoin.withColumn("rankTotal", dense_rank().over(Window.orderBy(desc("Indice_Total"))))
dfjoin = dfjoin.withColumn("Media",round(dfjoin['avg(Suma)'],2))
dfjoin.printSchema()

root
 |-- COLE_COD_DANE_ESTABLECIMIENTO: string (nullable = true)
 |-- COLE_NOMBRE_ESTABLECIMIENTO: string (nullable = true)
 |-- avg(Suma): double (nullable = true)
 |-- num_est: long (nullable = false)
 |-- Indice_Prom: double (nullable = true)
 |-- Indice_Num: double (nullable = true)
 |-- Indice_Total: double (nullable = true)
 |-- rankProm: integer (nullable = true)
 |-- rankNum: integer (nullable = true)
 |-- rankTotal: integer (nullable = true)
 |-- Media: double (nullable = true)



In [263]:
# Se ordena por Indice_Prom
dfjoin = dfjoin.orderBy('rankProm',ascending= True)
dfjoin.select('COLE_NOMBRE_ESTABLECIMIENTO','Media','Indice_Prom','num_est','rankProm','rankNum','rankTotal').show(10)

+---------------------------+------+-----------+-------+--------+-------+---------+
|COLE_NOMBRE_ESTABLECIMIENTO| Media|Indice_Prom|num_est|rankProm|rankNum|rankTotal|
+---------------------------+------+-----------+-------+--------+-------+---------+
|       COL LA QUINTA DEL...|586.67|        1.0|     21|       1|    307|      215|
|            COL LOS NOGALES| 583.2|      0.994|     50|       2|    282|      190|
|       COL SAN JORGE DE ...|578.47|      0.986|     66|       3|    269|      178|
|       COLEGIO BILINGUE ...|570.75|      0.973|     12|       4|    314|      222|
|       COLEGIO HISPANOAM...|569.67|      0.971|     76|       5|    261|      171|
|       INST ALBERTO MERA...|567.69|      0.968|     29|       6|    300|      209|
|       COLEGIO COLOMBO B...|566.55|      0.966|     71|       7|    265|      175|
|            COLEGIO FREINET|565.97|      0.965|     33|       8|    297|      206|
|       COL  SAN BONIFACI...|565.76|      0.964|     21|       9|    307|   

In [264]:
# Se ordena por Indice_Num
dfjoin = dfjoin.orderBy('rankNum',ascending= True)
dfjoin.select('COLE_NOMBRE_ESTABLECIMIENTO','Media','Indice_Num','num_est','rankProm','rankNum','rankTotal').show(10)

+---------------------------+------+----------+-------+--------+-------+---------+
|COLE_NOMBRE_ESTABLECIMIENTO| Media|Indice_Num|num_est|rankProm|rankNum|rankTotal|
+---------------------------+------+----------+-------+--------+-------+---------+
|       COLEGIO TECNISIST...|369.12|       1.0|   1202|     316|      1|        2|
|             INST EDUC CEFA|419.01|     0.909|   1093|     231|      2|        1|
|       CENT JOHANN KEPLE...|355.59|     0.895|   1076|     339|      3|        3|
|       INST EDUC INEM JO...|417.49|     0.691|    831|     233|      4|        4|
|       I. E. ESCUELA NOR...|367.34|     0.549|    660|     319|      5|        7|
|       LIC SALAZAR Y HER...|441.85|     0.548|    659|     192|      6|        5|
|       INSTITUCION EDUCA...| 395.7|     0.516|    620|     271|      7|        6|
|       I. E. LICEO CAUCASIA|365.95|     0.494|    594|     321|      8|       12|
|       IE TEC INEM CUSTO...|398.65|     0.488|    586|     265|      9|        8|
|   

In [265]:
# Se ordena por Indice_Total
dfjoin = dfjoin.orderBy('rankTotal',ascending= True)
dfjoin.select('COLE_NOMBRE_ESTABLECIMIENTO','Media','Indice_Total','num_est','rankProm','rankNum','rankTotal').show(10)

+---------------------------+------+------------+-------+--------+-------+---------+
|COLE_NOMBRE_ESTABLECIMIENTO| Media|Indice_Total|num_est|rankProm|rankNum|rankTotal|
+---------------------------+------+------------+-------+--------+-------+---------+
|             INST EDUC CEFA|419.01|       0.649|   1093|     231|      2|        1|
|       COLEGIO TECNISIST...|369.12|       0.629|   1202|     316|      1|        2|
|       CENT JOHANN KEPLE...|355.59|       0.542|   1076|     339|      3|        3|
|       INST EDUC INEM JO...|417.49|       0.492|    831|     233|      4|        4|
|       LIC SALAZAR Y HER...|441.85|       0.413|    659|     192|      6|        5|
|       INSTITUCION EDUCA...| 395.7|       0.348|    620|     271|      7|        6|
|       I. E. ESCUELA NOR...|367.34|       0.344|    660|     319|      5|        7|
|       IE TEC INEM CUSTO...|398.65|       0.332|    586|     265|      9|        8|
|       COLEGIO NACIONAL ...|412.38|       0.315|    539|     242

## Colegios

In [105]:
# SE crea Dataframe de colegios en este año desaparecio la columna COLE_CODMPIO_COLEGIO
dfc = spark.read.load(base + '/' + coleDic['2012'],"com.databricks.spark.csv",header='true',inferSchema='true',sep='|')
dfc = dfc.withColumn('COLE_CODIGO_COLEGIO', dfc['COLE_CODIGO_COLEGIO'].cast("string"))

In [106]:
print(dfc.printSchema())

root
 |-- COLE_CODIGO_COLEGIO: string (nullable = true)
 |-- COLE_INST_NOMBRE: string (nullable = true)
 |-- COLE_CODMPIO_COLEGIO: integer (nullable = true)
 |-- COLE_MPIO_COLEGIO: string (nullable = true)
 |-- COLE_DEPTO_COLEGIO: string (nullable = true)
 |-- COLE_INST_JORNADA: string (nullable = true)
 |-- COLE_CALENDARIO_COLEGIO: string (nullable = true)
 |-- COLE_GENEROPOBLACION: string (nullable = true)
 |-- COLE_NATURALEZA: string (nullable = true)
 |-- COLE_GEOGRAFIA: string (nullable = true)
 |-- COLE_QUIMICA: integer (nullable = true)
 |-- COLE_FISICA: integer (nullable = true)
 |-- COLE_BIOLOGIA: integer (nullable = true)
 |-- COLE_HISTORIA: string (nullable = true)
 |-- COLE_FILOSOFIA: integer (nullable = true)
 |-- COLE_MATEMATICAS: integer (nullable = true)
 |-- COLE_LENGUAJE: integer (nullable = true)
 |-- COLE_CIENCIAS_SOCIALES: integer (nullable = true)
 |-- COLE_INGLES: integer (nullable = true)
 |-- COLE_CATEGORIA: string (nullable = true)
 |-- CLPL_N: integer (nullab

In [107]:
colsColegio = ['COLE_CODIGO_COLEGIO',
 'COLE_CIENCIAS_SOCIALES',
 'COLE_QUIMICA',
 'COLE_FISICA',
 'COLE_BIOLOGIA',
 'COLE_FILOSOFIA',
 'COLE_MATEMATICAS',
 'COLE_LENGUAJE',
 'COLE_INGLES',
 'COLE_GEOGRAFIA',
 'COLE_HISTORIA']

In [108]:
dfc = dfc.select(colsColegio)

In [109]:
dfc.show(5,False)

+-------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|COLE_CODIGO_COLEGIO|COLE_CIENCIAS_SOCIALES|COLE_QUIMICA|COLE_FISICA|COLE_BIOLOGIA|COLE_FILOSOFIA|COLE_MATEMATICAS|COLE_LENGUAJE|COLE_INGLES|COLE_GEOGRAFIA|COLE_HISTORIA|
+-------------------+----------------------+------------+-----------+-------------+--------------+----------------+-------------+-----------+--------------+-------------+
|69278              |8                     |8           |8          |8            |8             |8               |8            |9          |null          |null         |
|69336              |7                     |7           |7          |6            |6             |7               |6            |6          |null          |null         |
|69351              |7                     |8           |8          |7            |6             |8               |7            |7          |null

In [110]:
print("El numero de colegios es: ", dfc.select('COLE_CODIGO_COLEGIO').count())

El numero de colegios es:  8359


## Anàlisis comparativo entre estudiantes y colegios

In [111]:
dfjoin = df1.join(dfc, df1.COLE_COD_DANE_ESTABLECIMIENTO == dfc.COLE_CODIGO_COLEGIO)

In [112]:
print("Nùmero de colegios coincidentes entre el archivo de estudiantes:", dfjoin.count())

Nùmero de colegios coincidentes entre el archivo de estudiantes: 0


La informaciòn de colegios no coincide con la informaciòn de estudiantes, no existe còdigo dane en el archivo de colegios.